<a href="https://colab.research.google.com/github/MayankDey20/Encryption-and-Decryption/blob/main/Python_Encryption_%26_Decryption_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

# pip install gradio cryptography

import gradio as gr
from cryptography.fernet import Fernet
import os
import tempfile

def process_message(action, message, key_file_obj):
    """
    A single function to handle both encryption and decryption.
    """
    # --- Input Validation ---
    if key_file_obj is None:
        raise gr.Error("A key file is required. Please upload your '.key' file.")

    if not message:
        raise gr.Error("A message is required. Please enter text to process.")

    # --- Key Loading ---
    try:
        with open(key_file_obj.name, "rb") as f:
            key = f.read()
        f = Fernet(key)
    except Exception:
        raise gr.Error("Invalid key file. Please ensure it's a valid key generated by this tool.")

    # --- Action ---
    if action == "Encrypt":
        try:
            encrypted_message = f.encrypt(message.encode())
            return encrypted_message.decode()
        except Exception as e:
            raise gr.Error(f"Encryption failed: {e}")

    elif action == "Decrypt":
        try:
            decrypted_message = f.decrypt(message.encode())
            return decrypted_message.decode()
        except Exception:
            # Avoid leaking detailed crypto errors.
            raise gr.Error("Decryption failed. The message may be invalid, tampered with, or the wrong key was used.")

def generate_key_file():
    """
    Generates a new Fernet key and provides it as a downloadable file.
    """
    key = Fernet.generate_key()

    # Create a temporary file to hold the key for downloading.
    # Gradio will handle the cleanup.
    with tempfile.NamedTemporaryFile(delete=False, suffix=".key", mode="wb") as temp_key_file:
        temp_key_file.write(key)
        return temp_key_file.name

# --- Gradio UI Layout using Blocks ---
with gr.Blocks(theme=gr.themes.Soft(primary_hue="slate", secondary_hue="blue"), title="Secure Encrypter/Decrypter") as app:
    gr.Markdown(
        """
        # 🔑 Secure Encrypter & Decrypter
        A simple and secure tool to encrypt and decrypt your messages.
        Uses Fernet (AES128-CBC) for symmetric encryption.
        """
    )

    with gr.Tabs():
        with gr.TabItem("Encrypt / Decrypt"):
            with gr.Row():
                with gr.Column(scale=2):
                    # Input for the message
                    text_input = gr.Textbox(lines=5, label="Input Message", placeholder="Type your secret message here...")

                    # Radio buttons to choose the action
                    action_choice = gr.Radio(["Encrypt", "Decrypt"], label="Action", value="Encrypt")

                    # Button to trigger the process
                    submit_btn = gr.Button("Process Message", variant="primary")

                with gr.Column(scale=1):
                    # File uploader for the key
                    gr.Markdown("### Upload Your Key")
                    key_file_input = gr.File(label="Upload your '.key' file", file_types=[".key"])

            # Output for the result
            gr.Markdown("---")
            gr.Markdown("### 📝 Result")
            text_output = gr.Textbox(lines=5, label="Output", interactive=False)

        with gr.TabItem("Generate New Key"):
            gr.Markdown(
                """
                ## Generate a New Encryption Key
                Click the button below to generate a new `secret.key` file.
                **Important:** Keep this key file safe! Anyone who has it can decrypt your messages.
                """
            )
            # Button to generate a new key
            generate_btn = gr.Button("Generate & Download Key", variant="primary")

            # File component to handle the download
            key_file_output = gr.File(label="Download your new key file", interactive=False)

    # --- Event Handling ---
    submit_btn.click(
        fn=process_message,
        inputs=[action_choice, text_input, key_file_input],
        outputs=text_output
    )

    generate_btn.click(
        fn=generate_key_file,
        inputs=None,
        outputs=key_file_output
    )

# Launch the Gradio app
if __name__ == "__main__":
    app.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://06cf845f8de44c4be9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
